In [2]:
import numpy as np
from numpy.random import uniform, multivariate_normal, dirichlet
from numpy.linalg import norm
from gatherMatrices import getMixtureEmissionMatrices
import pickle

np.set_printoptions(linewidth=100000)

with open("../data/allDF.pickle", "rb") as f:
    df = pickle.load(f)

emissionMatrix = getMixtureEmissionMatrices(df)

In [66]:
def create_uniform_particles(
        a_range, f_range, s_range,
        n_range, i_range,
        l_range, j_range,
        N
):
    particles = np.empty((N, 7))
    for i, (var, range) in enumerate(locals().items()):
        if var[-5:] != "range": break
        particles[:, i] = uniform(range[0], range[1], size=N)
    for i, row in enumerate(particles):
        particles[i, :] = row / sum(row)

    return particles

In [67]:
def predict(particles, phi, epsilon_var):
    """
    Move alpha forward one time step according to alpha_t = phi * alpha_t-1 + epsilon
    Note: this operation is in-place
    """
    for i, alpha_t_1 in enumerate(particles):
        epsilon = multivariate_normal(
            mean=[0 for _ in range(len(alpha_t_1))],
            cov=np.eye(len(alpha_t_1)) * epsilon_var
        )
        particles[i, :] = np.matmul(phi, alpha_t_1.T).squeeze() + epsilon


In [ ]:
def estimateParticleWeight(alpha: np.array, harmony: int, N=50) -> float:
    """
    Approximate the weight of a given particle associated with a given harmony
    :param alpha: the alpha value associated with the particle
    :param harmony: the observed harmony
    :param N: number of emission particles used to estimate the transition particle weights
    :return: Estimated weight of the given particle
    """
    samples = []
    for _ in range(N):
        p = dirichlet(alpha)
        h_dist = np.matmul(p, emissionMatrix).squeeze()
        h_dist = h_dist / norm(h_dist, ord=1)
        samples.append(h_dist[harmony]) #TODO: check that harmony index is same throughout
    return sum(samples) / len(samples)


def update(particles, harmony, weights):
    for i, particle in enumerate(particles):
        weights[i] *= estimateParticleWeight(particle, harmony, 50)
    weights += 1e-300
    weights /= sum(weights)

In [69]:
particles = create_uniform_particles(
    (0, 1),
    (0, 1),
    (0, 1),
    (0, 1),
    (0, 1),
    (0, 1),
    (0, 1),
    4
)
print(particles)
predict(particles, np.eye(7), 0.01)
print(particles)

[[0.23872624 0.23963555 0.07825378 0.00709433 0.18740726 0.1242051  0.12467774]
 [0.23128638 0.03644153 0.14091685 0.19241192 0.18806878 0.19888903 0.0119855 ]
 [0.12836095 0.0131849  0.05941033 0.02482454 0.10896621 0.34926902 0.31598404]
 [0.20518802 0.09750528 0.12424497 0.12532539 0.10431097 0.18986809 0.15355727]]
[[ 0.33677835  0.10808977  0.14443352 -0.21392147  0.20637276  0.28013079  0.23895313]
 [ 0.32933998 -0.02085002  0.28734317  0.27429818  0.06815903 -0.02813378  0.13656152]
 [ 0.16267748 -0.08834668 -0.00616265 -0.09097869  0.09088773  0.3003593   0.34493424]
 [ 0.17848427  0.08488015 -0.0625159   0.20672036  0.11266026  0.20676925  0.08006055]]
